Dependencies

In [2]:
import numpy as np
import pandas as pd
import open3d as o3d
import transforms3d as tf3d
import csv
from pyntcloud import PyntCloud as pt
import os


__Data Processing Steps__

In [3]:
def data_array_from_column(df, column_name):
    vectors = df[column_name].apply(eval)
    vectors_df = pd.DataFrame(vectors.tolist())
   

    return np.asarray(vectors_df)

File path

In [4]:
relative_path = '/home/shubham/Master_Thesis/label_data/'
seed_paths = os.listdir('/home/shubham/Master_Thesis/label_data')
pcd_files = os.listdir(relative_path+seed_paths[0]+"/pointclouds")
csv_file = relative_path+seed_paths[0]+"/output.csv"

Data Extractions and cleanup

In [5]:
data = pd.read_csv(csv_file)
data['SimTime'] = np.round(data['SimTime'], 2)
data.drop('WallTime', inplace=True, axis=1)
data.drop('PropertyType', inplace=True, axis=1)
data.drop('InstanceType', inplace=True, axis=1)
data.drop('ModelInstanceIdPath', inplace=True, axis=1)


In [79]:
data

,SimTime,PropertyName,InstanceName,Data
0,0.0,childsizes,road1_bb,"[(0.26500000000000001,0.016,0.2650000000000000..."
1,0.0,childpositions,road1_bb,"[(304.99023161931717,336.71550169570912,0.3049..."
2,0.0,childorientations,road1_bb,"[(1.7791611020322677,0,0),(1.7791886394830212,..."
3,0.0,childmodelinstances,road1_bb,[]
4,0.0,childsizes,road2_bb,"[(0.68799999999999994,0.23200000000000001,0.68..."
...,...,...,...,...
9875,75.8,childorientations,sumo_bb,"[(2.3892946594564179,0,0),(0,0,0),(0,0,0),(0,0..."
9876,75.8,childpositions,sumo_bb,"[(14.229153768199685,691.9462474352365,2.25),(..."
9877,75.8,childsizes,sumo_bb,"[(4.9800000000000004,2,0.5),(4.980000000000000..."
9878,75.8,worldFrame,simulated_lidar,"(0.67626447004604251,-0.7366561687387565,-0.00..."


In [22]:
childsizes_all = data.loc[data['PropertyName']=='childsizes']
childsizes_array = data_array_from_column(childsizes_all,'Data')
childpositions_all = data.loc[data['PropertyName']=='childpositions']
childpositions_array = data_array_from_column(childpositions_all,'Data')
childorientations_all = data.loc[data['PropertyName']=='childorientations']
childorientations_array = data_array_from_column(childorientations_all,'Data')

In [81]:
childpositions_array[5].shape

(36,)

In [30]:
childsizes_filtered = []
childpositions_filtered = []
for i in range(childorientations_array.shape[0]):
        childsizes_array_temp = [x for x in childsizes_array[i] if x is not None]
        childpositions_array_temp = [x for x in childpositions_array[i] if x is not None]   
        childpositions_filtered.append(childpositions_array_temp)
        childsizes_filtered.append(childsizes_array_temp) 

In [82]:
childpositions_filtered[5]

[(363.4576812763993, 304.1968222250635, 2.25),
 (179.7813495047468, 153.62411627817406, 1.3716034877949148),
 (127.73019067836991, 200.59545822912656, 1.3361612839987342),
 (133.73019067836992, 200.59545822912656, 1.3361612839987342),
 (139.73019067836992, 200.59545822912656, 1.3361612839987342),
 (145.73019067836992, 200.59545822912656, 1.3361612839987342),
 (151.73019067836992, 200.59545822912656, 1.3361612839987342),
 (157.73019067836992, 200.59545822912656, 1.3361612839987342),
 (163.73019067836992, 200.59545822912656, 1.3361612839987342),
 (169.73019067836992, 200.59545822912656, 1.3361612839987342),
 (175.73019067836992, 200.59545822912656, 1.3361612839987342),
 (181.73019067836992, 200.59545822912656, 1.3361612839987342),
 (121.73019067836991, 200.59545822912656, 1.3361612839987342),
 (127.73019067836991, 200.59545822912656, 1.3361612839987342),
 (133.73019067836992, 200.59545822912656, 1.3361612839987342),
 (139.73019067836992, 200.59545822912656, 1.3361612839987342),
 (145.730

BB calculation

In [55]:
BB_length = []
BB_width = []
BB_height = []
BB_center = []

for i in range(len(childsizes_filtered)):
    for j in range(len(childsizes_filtered[i])):
        length = (childsizes_filtered[i][j][0])/2
        width = (childsizes_filtered[i][j][1])/2
        height = (childsizes_filtered[i][j][2])/2
        center = childpositions_filtered[i][j]
        
        BB_length.append(length)
        BB_width.append(width)
        BB_height.append(height)
        BB_center.append(center)

In [51]:
total_objects = len(BB_length)
total_frames = int(len(childsizes_array)/6) # 6 is the number of times childsize is calculated in one time frame
total_unique_objects_per_frame = int(total_objects/total_frames)

Reshape the BB dimensions

In [56]:
BB_center = np.reshape(BB_center, (total_frames, total_unique_objects_per_frame, 3))
BB_length = np.reshape(BB_length, (total_frames, total_unique_objects_per_frame))
BB_width = np.reshape(BB_width, (total_frames, total_unique_objects_per_frame))
BB_height = np.reshape(BB_height, (total_frames, total_unique_objects_per_frame))

TF for camera and lidar

In [7]:
tf_camera_data = data.loc[data['InstanceName']=='Simulated Front Camera (internal reference frame)']
tf_camera_array = data_array_from_column(tf_camera_data,'Data')

tf_lidar_data = data.loc[data['InstanceName']=='simulated_lidar']
tf_lidar_array = data_array_from_column(tf_lidar_data,'Data')

V2C matrix

In [62]:
V2C_array = []
inv_lidar_tf_array = []
for i in range(len(tf_camera_array)):
    inv_lidar_tf = np.linalg.inv(tf_lidar_array[i].reshape(4,4))
    V2C = np.dot(inv_lidar_tf,tf_camera_array[i].reshape(4,4))
    V2C_array.append(V2C)
    inv_lidar_tf_array.append(inv_lidar_tf)


Transform BB center from world to lidar coordinates

In [73]:
position_from_lidar = []
for i in range(total_frames):
    for j in range(total_unique_objects_per_frame):
        positionA = np.reshape(BB_center[i][j][:3],(3,1))
        RotAB = inv_lidar_tf_array[i][:3,:3]
        translationAB = np.reshape(inv_lidar_tf_array[i][:3,3],(3,1))
        alpha = np.dot(RotAB,positionA)
        beta = alpha + translationAB
        position_from_lidar.append(beta)
    
    
position_from_lidar = np.reshape(position_from_lidar, (total_frames, total_unique_objects_per_frame, 3))

In [74]:
position_from_lidar[0]

array([[ 1.25337934e+01,  6.48556586e+01, -1.95111389e+00],
       [ 1.65900131e+01,  6.80222001e+01, -1.95111389e+00],
       [ 2.06136877e+01,  7.06136097e+01, -1.95111389e+00],
       [ 2.43824772e+01,  7.29024889e+01, -1.95111389e+00],
       [ 3.15296917e+01,  7.72448887e+01, -1.89782672e+00],
       [ 3.61839672e+01,  7.97945934e+01, -1.73754814e+00],
       [ 4.07041660e+01,  8.25269504e+01, -1.82417384e+00],
       [ 4.49449937e+01,  8.55712257e+01, -1.82417384e+00],
       [ 4.93903648e+01,  8.86421360e+01, -1.82417384e+00],
       [ 5.34568310e+01,  9.18344366e+01, -1.82417384e+00],
       [ 7.00457474e+01,  1.04869594e+02, -1.82417384e+00],
       [ 7.62463282e+01,  1.10401065e+02, -1.58353946e+00],
       [ 8.05965193e+01,  1.14312861e+02, -1.58353946e+00],
       [ 8.86991793e+01,  1.22021532e+02, -1.58353946e+00],
       [ 9.39952763e+01,  1.27655397e+02, -1.58353946e+00],
       [ 9.90222890e+01,  1.32945305e+02, -1.58353946e+00],
       [ 1.03541824e+02,  1.37335942e+02

array([[304.99046214, 336.19310651, -30.36525842],
       [303.25635676, 341.01835258, -30.8008983 ],
       [302.05813549, 345.63311627, -31.21753095],
       [301.06720502, 349.91231526, -31.60386687],
       [299.18634984, 358.03280985, -32.2834999 ],
       [298.22556571, 363.24529171, -32.5931621 ],
       [297.04928925, 368.36585305, -33.14243607],
       [295.48919449, 373.32747336, -33.59038627],
       [293.96798105, 378.49084583, -34.05655035],
       [292.21263178, 383.33383038, -34.49379185],
       [285.04005876, 403.0943678 , -36.27784206],
       [281.73317644, 410.7080451 , -36.72362327],
       [279.38367127, 416.04415198, -37.20538932],
       [274.60625497, 426.11521225, -38.11464989],
       [270.91838567, 432.88390745, -38.7257624 ],
       [267.47268641, 439.29066631, -39.30419704],
       [264.72164503, 444.93653437, -39.81393203],
       [194.98250685, 535.78455501, -48.34653475],
       [189.31931506, 540.26763319, -48.75131912],
       [184.41841121, 544.85754

Filter out the BBs which are out of Lidar context

In [68]:
Lidar_clipping_range = [0, -40, -3, 70.4, 40, 1]
for i in range(total_frames):
    for idx, j in enumerate(position_from_lidar[i]):
        if j[0] > Lidar_clipping_range[3] or j[0] < Lidar_clipping_range[0] or j[1] > Lidar_clipping_range[4] or j[1] < Lidar_clipping_range[1] or j[2] > Lidar_clipping_range[5] or j[2] < Lidar_clipping_range[2]:
            position_from_lidar[i][idx] = np.array([0, 0, 0])
   

(380, 73, 3)

Calculate Heading angle of the cars

In [11]:
data_array = childorientations_array


In [18]:
childpositions_array.shape

(2280, 36)

Convert from Radians to Degrees and find relative angles

In [5]:
converted_angles = []

for i in range(len(childorientations_array)):
    for j in range(len(childorientations_array[0])):
        for k in range(len(childorientations_array[0][0])):
            radian_values = (childorientations_array[i][j][k])*57.2957795
            converted_angles.append(radian_values)


converted_angles = np.reshape(converted_angles,(6,73,3))

In [6]:
only_roll = []

for i in range(len(converted_angles)):
    for j in range(len(converted_angles[0])):
        roll = converted_angles[i][j][0]
        only_roll.append(roll)

only_roll = np.reshape(only_roll,(6,73))

In [7]:
reference_value = [i[0]for i in only_roll]
allothervalues =  np.delete(only_roll,0,axis=1)
relative_values = []
for i in range(len(allothervalues)):
    rel_value = allothervalues[i] - reference_value[i]
    relative_values.append(rel_value)



Transformation Matrix


In [8]:
tf_data = pd.read_csv("tfdata.csv")
tf_array = data_array_from_column(tf_data,'Data')

Create Pose of the Objects

In [9]:
childPose = []
for i in range(len(childpositions_array)):
    for j in range(len(childpositions_array[0])):
        arr1= childpositions_array[i][j]
        arr2 = childorientations_array[i][j]
        temp_array = np.concatenate((arr1, arr2 ), axis=0)
        childPose.append(temp_array)

childPose = np.asarray(childPose)   
childPose = childPose.reshape(6,73,6)


Transform from World Frame to Vehicle Frame

In [10]:
new_worldframes = []
for i in range(len(tf_array)):
    temp_array = np.asarray(tf_array[i])
    temp_array = temp_array.reshape(4,4)
    new_worldframes.append(temp_array)

In [11]:
inverse_tf = []
for i in range(len(new_worldframes)):
    inv_tf = np.linalg.inv(new_worldframes[i])
    inverse_tf.append(inv_tf)

In [12]:
transformed_position =[]
for i in range(len(childPose)):
    for j in range(len(childPose[0])):
        positionA = np.reshape(childPose[i][j][:3],(3,1))
        RotAB = inverse_tf[i][:3,:3]
        translationAB = np.reshape(inverse_tf[i][:3,3],(3,1))
        alpha = np.dot(RotAB,positionA)
        beta = alpha + translationAB
        transformed_position.append(beta)


transformed_position = np.reshape(transformed_position,(6,73,3))

Create  bounding box edge coordinates

In [13]:
BB_length = []
BB_width = []
BB_height = []

for i in range(len(childsizes_array)):
    for j in range(len(childsizes_array[0])):
        length = (childsizes_array[i][j][0])/2
        width = (childsizes_array[i][j][1])/2
        height = (childsizes_array[i][j][2])/2
        
        BB_length.append(length)
        BB_width.append(width)
        BB_height.append(height)

BB_length = np.reshape(BB_length,(6,73))
BB_height = np.reshape(BB_height,(6,73))
BB_width = np.reshape(BB_width,(6,73))

In [14]:
BB_coordinates = []

for i in range(len(BB_height)):
    for j in range(len(BB_height[0])):
        x = transformed_position[i][j][0]
        y = transformed_position[i][j][1]
        z =  transformed_position[i][j][2]

        l = BB_length[i][j]
        h = BB_height[i][j]
        w = BB_width[i][j]

        edges = []
        e1 = (x-w,y-l,z-h)
        e2 = (x-w,y-l,z+h)
        e3 = (x-w,y+l,z+h)
        e4 = (x-w,y+l,z-h)
        e5 = (x+w,y-l,z-h)
        e6 = (x+w,y-l,z+h)
        e7 = (x+w,y+l,z+h)
        e8 = (x+w,y+l,z-h)
        edges.append(e1)
        edges.append(e2)
        edges.append(e3)
        edges.append(e4)
        edges.append(e5)
        edges.append(e6)
        edges.append(e7)
        edges.append(e8)

        BB_coordinates.append(edges)

BB_coordinates = np.reshape(BB_coordinates,(6,73,8,3))

Save as csv

In [15]:
fil_name = 'BB_coordinates'
with open(fil_name+'.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(BB_coordinates)

Testing one Object from timestep 0.2


In [16]:
Obj_BB_Coord = BB_coordinates[1][1]
loc = "C:\\Users\\Proteus\\Desktop\\RWTH\\HiWi\\UrbanTestDrive\\pcd\\test.pcd_simTime-199.99917603ms.pcd"
pc_read = o3d.io.read_point_cloud(loc)
pcd_data = np.asarray(pc_read.points)


Draw PCD

In [17]:
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pcd_data)
o3d.io.write_point_cloud("pcd1.ply",pcd1)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(Obj_BB_Coord)
o3d.io.write_point_cloud("pcd2.ply",pcd2)

pcd1_read = o3d.io.read_point_cloud("pcd1.ply")
pcd2_read = o3d.io.read_point_cloud("pcd2.ply")

pcd1_read.paint_uniform_color([0,0,0.75]) # choose a unique rgb value for each point cloud
pcd2_read.paint_uniform_color([0,0.5,0])

bb_data = o3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd2.points)
obb_data = o3d.geometry.AxisAlignedBoundingBox.get_oriented_bounding_box(bb_data)
# o3d.visualization.draw_geometries([pcd1_read,pcd2_read])
o3d.visualization.draw_geometries([pcd1_read,obb_data])